In [16]:
import numpy as np
import os
import pandas as pd
from scipy import signal
import pickle
from scipy.signal import chirp, find_peaks, peak_widths
import ephyviewer
from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter
from ephyviewer import InMemoryAnalogSignalSource
import matplotlib.pyplot as plt

### Load and visualise dose-resp recording LFPs to find out if spindles are up or down (+/-)

In [22]:
date_time = "2024-08-16_15-23-39"
Letter = "F:"
nb_E = 15

stim = np.load(f"{Letter}/{date_time}/Record Node 101/experiment1/recording1/events/Acquisition_Board-100.Rhythm Data/TTL/sample_numbers.npy")
channel_states = np.load(f"{Letter}/{date_time}/Record Node 101/experiment1/recording1/events/Acquisition_Board-100.Rhythm Data/TTL/states.npy")
cont = np.load(f"{Letter}/{date_time}/Record Node 101/experiment1/recording1/continuous/Acquisition_Board-100.Rhythm Data/sample_numbers.npy")

All = np.load('C:/Users/NOG-Analysis/Documents/Python/Annie/Mouse Recordings - Annie/23__/2024-08-16_15-23-39/RawDataChannelExtractedDS_1.npy', mmap_mode= 'r')

EMG = All[:,0]
y = 1
for x in range(1, nb_E+1):
    exec(f"Ea = All[:,{y}]")
    y += 1
    exec(f"Eb = All[:,{y}]")
    y += 1
    exec(f"E{x} = Ea - Eb")

### Create vectors to display stims
ttls = [1,2]
for TTL in ttls:
    # find the indices of the start and end times
    start_indices = np.where(channel_states == TTL)[0]
    end_indices = np.where(channel_states == -TTL)[0]

    # create a new array with the start and end times
    stim_times = np.zeros((len(start_indices), 2))
    for i in range(len(start_indices)):
        stim_times[i, 0] = stim[start_indices[i]]
        stim_times[i, 1] = stim[end_indices[i]]

    #Adjust stim timestamps - start time and sampling frequency
    start_time = cont[0]  # in 0.5 milliseconds   #First value of timestamps memmap from 'continuous' folder
    print(f'start time = {start_time} in 0.5 ms or {start_time/(2000*60)} in min')
    stim_times -= start_time
    stim_times = stim_times.astype('float64')
    stim_times /= 2.0
    stim_times = stim_times.astype('int64')

    # Assigning values stim (3) and non stim (0)
    display_stim = np.zeros(EMG.size)
    for x in stim_times:
        display_stim[x[0]:x[1]] = 3
    globals()[f"display_stim_{TTL}"] = display_stim



# Afficher 15 LFP et stims
app = mkQApp()

combined = np.stack([EMG, E1, E2, E3, E4, E5, E6, E7, E8, E9, E10, E11, E12, E13, E14, E15, display_stim_1, display_stim_2], axis = 1)

sample_rate = 1000.
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer()

view1 = TraceViewer.from_numpy(combined, sample_rate, t_start, 'Signals')

view1.params['display_labels'] = True

normaliser = [np.mean(EMG**2)]
for x in range(1, nb_E+1):
    exec(f"Ele = E{x}")
    nor = np.mean(Ele**2)
    normaliser.append(nor)

for i in range(16):
    channel = f'ch{i}'
    view1.by_channel_params[channel, 'gain'] = 150/normaliser[i]
    view1.by_channel_params[channel, 'offset'] = 20 - 2*(i+1)

view1.by_channel_params['ch16', 'gain'] = 0.5
view1.by_channel_params['ch17', 'gain'] = 0.5
view1.by_channel_params['ch16', 'offset'] = -16
view1.by_channel_params['ch17', 'offset'] = -18

#add them to mainwindow
win.add_view(view1)

#show main window and run Qapp
win.show()
app.exec()

start time = 0 in 0.5 ms or 0.0 in min
start time = 0 in 0.5 ms or 0.0 in min


0

### Compute phase difference for all stims in all recordings, store in dataframe dictionary

In [ ]:
#Variables that might change from mouse to mouse

# format: recordings = [date_time, TTL of midline, TTL of VB]       #put float("nan") if no stim of this type in recording!!
recordings = [["2024-08-21_16-53-14", 2], ["2024-08-23_13-07-15", 1], ["2024-09-12_14-57-36", 2]]
#recordings = [["2024-08-21_16-53-14", 2]]

stim_regions = ["mid"]    #make sure to have that in same order as TTLs in recordings info
Letter = "F:"
nb_E = 15
mouse = os.path.basename(os.getcwd())   #script contained in folder named after mouse

spindles_info_list = []

for rec_nb, rec_infos in enumerate(recordings):
    print(rec_infos)
    date_time = rec_infos[0]
    if date_time == "2024-09-12_14-57-36":
        recordnode = 112
        acquiboard = 111
    else:
        recordnode = 101
        acquiboard = 100

    ### STIM TIMES ###
    #Load stim info from hard drive with original recording
    sample_numbers = np.load(f"{Letter}/{date_time}/Record Node {recordnode}/experiment1/recording1/events/Acquisition_Board-{acquiboard}.Rhythm Data/TTL/sample_numbers.npy")
    channel_states = np.load(f"{Letter}/{date_time}/Record Node {recordnode}/experiment1/recording1/events/Acquisition_Board-{acquiboard}.Rhythm Data/TTL/states.npy")
    cont = np.load(f"{Letter}/{date_time}/Record Node {recordnode}/experiment1/recording1/continuous/Acquisition_Board-{acquiboard}.Rhythm Data/sample_numbers.npy")

    for pos, stim_region in enumerate(stim_regions):
        TTL = rec_infos[pos+1]  #because loops through mid then vb, and info about TTL is in position 1 and 2 of sub-list in "recordings" (line 4)

        # find the indices of the start and end times
        start_indices = np.where(channel_states == TTL)[0]
        end_indices = np.where(channel_states == -TTL)[0]

        # create a new array with the start and end times
        stim_times = np.column_stack((sample_numbers[start_indices], sample_numbers[end_indices]))

        #Adjust stim timestamps - start time and sampling frequency
        start_time = cont[0]  # in 0.5 milliseconds   #First value of timestamps memmap from 'continuous' folder
        stim_times = ((stim_times - start_time).astype('float64') / 2.0).astype('int64')
        
    del sample_numbers; del channel_states; del cont; del start_indices; del end_indices




    ### LFP RECORDINGS ###
    # Load DS and ordered recordings, and sleep scoring frame, from subfolder
    EMGboolean = pd.read_pickle(f'{date_time}/EMGframeBoolean_1.pkl')
    All = np.load(f'{date_time}/RawDataChannelExtractedDS_1.npy', mmap_mode= 'r')

    #Get LFPs by calculating differences between electrode pairs (0-based)
    #EMG = All[:,0]
    electrodes = {}
    for x in range(nb_E):
        electrodes[x] = (All[:, 2*x+1] - All[:, 2*x+2]).astype(np.float32)   #replaces previous Ea-Eb; stores as float32 (half the memory usage)

    



    sleepstates = ['SWS']
    for sleepstate in sleepstates:
        w = 10.
        fs = 1000   #Sampling frequency
        freq = (np.arange(12, 18.5, 0.5)).astype(np.float32)
        widths = w*fs / (2*freq*np.pi)

        spindles_info_allCX = {}
        nb_E = 15
        for x in range(nb_E):   #finally switch to 0-based indexing
            ############################## UPDATE FOR EACH MOUSE ########################################
            if x in [0,1,3,6,7,10,13, 5,14]:    #6, 15 = CA1 so orientation doesn't matter, no spindles
                pol = 1
            elif x in [2,4,8,9,11,12]:
                pol = -1
            #############################################################################################
            Ele = electrodes[x]*pol

            E_cwt_complex = signal.cwt(Ele, signal.morlet2, widths, w=w)
            E_cwt_phase = (np.angle(E_cwt_complex).astype(np.float32))
            E_cwt_raw = np.absolute(E_cwt_complex)
            E_cwt_raw *= freq[:, np.newaxis]    #Ajuster pour diminution 1/f
            E_cwt_complex = None #Free memory

            CWT_wake0 = E_cwt_raw.copy()
            
            if sleepstate == "SWS":
                BooleanState = EMGboolean['BooleanLiberal']
            elif sleepstate == "REM":
                BooleanState = EMGboolean['REMSleep']
            CWT_wake0[:, ~BooleanState] = 0
            #Compute second mean CWT only for concerned sleep state (not all sleep)
            CWT_wakeremoved2 = E_cwt_raw.copy()
            E_cwt_raw = None
            CWT_wakeremoved2 = CWT_wakeremoved2[:, BooleanState]
            mean_CWT_wakeremoved2 = np.mean(CWT_wakeremoved2, axis=1)

            #Loop through stims and collect relevant data
            spindles_info_oneCX = []

            #all_stim_freq = (2,4,6,8,10,12,14,16,18,20,40,60,90)
            all_stim_freq = [14]
            for stim_freq in all_stim_freq:
                for stim_nb in range(0, len(stim_times), 10):   #every 10th stim because blocks of 10. To be optimised.
                    stim_start = stim_times[stim_nb, 0]
                    stim_end = stim_times[stim_nb+9, 1]
                    stim_len = round((9000/stim_freq)+10)  #same as stim_end-stim_start (IF MATCHING FREQ), just makes sure all stim blocs have the exact same length  ##BE REALLY CAREFUL ABOUT NOT CONFUSING ACTUAL AND CALCULATED STIM LENGTH
                    stim_phase = np.concatenate([np.tile((c := np.linspace(-np.pi, np.pi, round((stim_len - 10)/9), endpoint=False)), 9), c[:stim_len - len(c) * 9]])    #Fake phase vector assuming each stim pulse start is -pi, cycles up to pi[ before start of new pulse
                    itvl = CWT_wake0[:, stim_start:stim_start+stim_len]
                    if stim_freq<30:
                        if round(9000/(stim_end - stim_start-10)) == stim_freq and 0 != np.any(itvl):   
                            raw_spindle = Ele[stim_start:stim_start+stim_len]
                            phase = E_cwt_phase[:, stim_start:stim_start+stim_len]
                            maxfreqsindex = np.argmax(itvl, axis=0)
                            inst_phase = phase[maxfreqsindex, np.arange(len(maxfreqsindex))]    #Audrey's Igor method
                            phase_diff = (inst_phase - stim_phase + np.pi) % (2*np.pi) - np.pi   #Phase wrapped to [-pi,pi]
                            #Calculate circular mean of phase lag
                            unit_vectors = np.exp(1j * phase_diff)   # 1: Project onto unit circle using Euler's formula
                            mean_vector = np.mean(unit_vectors) # 2: Compute mean vector
                            mean_phase_diff = np.angle(mean_vector)  # 3: Get mean phase (angle)
                            vector_strength = np.abs(mean_vector)   # 4: Get magnitude (locking strength)
                            max_ampli = np.max(CWT_wakeremoved2, axis=0)    #instantaneous max amplitude across freqs
                            peak_zscore = (np.max(itvl)-np.mean(max_ampli))/np.std(max_ampli)
                            spindle_zscore = (np.mean(np.max(itvl, axis=0))-np.mean(max_ampli))/np.std(max_ampli)   #zscore of whole duration of spindle
                            mainfreq = freq[np.argmax(np.trapz(itvl, axis=1))]     #AUC method (trapz). Perform on all freqs of itvl, find index for correct freq

                            row = {
                                "raw_spindle": raw_spindle.astype(np.float32),
                                "itvl": itvl.astype(np.float32),
                                "phase": phase.astype(np.float32),
                                "inst_phase": inst_phase.astype(np.float32),
                                "phase_diff": phase_diff.astype(np.float32),
                                "mean_phase_diff": mean_phase_diff.astype(np.float32),
                                "vector_strength": vector_strength.astype(np.float32),
                                "peak_zscore": peak_zscore.astype(np.float32),
                                "spindle_zscore": spindle_zscore.astype(np.float32),
                                "mainfreq": mainfreq.astype(np.float32)
                                }

                            spindles_info_oneCX.append(row)

            spindles_info_oneCX = pd.DataFrame(spindles_info_oneCX) #turn list into dataframe after looping through all spindles
            spindles_info_allCX[x] = spindles_info_oneCX  #add dataframe to dictionary containing all Eles (all recording sites); key is Ele number (0-based index)
        spindles_info_list.append(spindles_info_allCX)  #append dict with info from one recording to list containing all recordings

# Initialize an empty output dictionary (to merge dicts from all recordings into one final dict)
spindles_info = {}

# Assume all dicts have the same site keys
for site in spindles_info_allCX.keys():
    # Collect the DataFrames from each dictionary for this site
    dfs = [d[site] for d in spindles_info_list]
    
    # Concatenate along rows (spindles)
    spindles_info[site] = pd.concat(dfs, ignore_index=True)

with open("Spindles_Phase_Info.pkl", "wb") as f:
    pickle.dump(spindles_info, f)    

['2024-08-21_16-53-14', 2]
['2024-08-23_13-07-15', 1]
['2024-09-12_14-57-36', 2]
